In [25]:
import logging 
# Set up basic logging 
logging.basicConfig( 

filename='etl_log.txt', 
level=logging.INFO, 
format='%(asctime)s - %(levelname)s - %(message)s' 
) 
import pandas as pd 
import numpy as np 
import mysql.connector 
import datetime 

In [26]:
csv_file_path = 'employees1.csv' 
df = pd.read_csv(csv_file_path) 
print("Raw data loaded:") 
print(df.head()) 
print(df.columns.tolist()) 
logging.info("CSV loaded successfully.")

Raw data loaded:
   EMPLOYEE_ID FIRST_NAME  LAST_NAME     EMAIL  PHONE_NUMBER  HIRE_DATE  \
0          198     Donald   OConnell  DOCONNEL  650.507.9833  21-Jun-07   
1          199    Douglas      Grant    DGRANT  650.507.9844  13-Jan-08   
2          200   Jennifer     Whalen   JWHALEN  515.123.4444  17-Sep-03   
3          201    Michael  Hartstein  MHARTSTE  515.123.5555  17-Feb-04   
4          202        Pat        Fay      PFAY  603.123.6666  17-Aug-05   

     JOB_ID  SALARY  
0  SH_CLERK    2600  
1  SH_CLERK    2600  
2   AD_ASST    4400  
3    MK_MAN   13000  
4    MK_REP    6000  
['EMPLOYEE_ID', 'FIRST_NAME', 'LAST_NAME', 'EMAIL', 'PHONE_NUMBER', 'HIRE_DATE', 'JOB_ID', 'SALARY']


In [27]:
df.fillna({ 
'EMAIL': 'not_provided@example.com', 
'PHONE_NUMBER': '0000000000',
'HIRE_DATE': '01-Jan-00', 
'SALARY': 0 
}, inplace=True)

In [28]:
df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]

In [29]:

print(df.columns.tolist())

['employee_id', 'first_name', 'last_name', 'email', 'phone_number', 'hire_date', 'job_id', 'salary']


In [30]:
df['hire_date'] = pd.to_datetime(df['hire_date'], format='%d-%b-%y', 
errors='coerce') 

In [31]:
# Replace invalid dates with a default 
df['hire_date'] = df['hire_date'].fillna(pd.to_datetime('2000-01-01')) 
# Replace non-numeric salaries with 0 
df['salary'] = pd.to_numeric(df['salary'], errors='coerce').fillna(0).astype(int) 

In [32]:
logging.info("Data cleaning completed.") 

In [33]:
mydb = mysql.connector.connect( 
    host="localhost", 
    user="root", 
    password="Giri#2004", 
    database="employee_1"   
) 

In [34]:
cursor = mydb.cursor() 

In [35]:
cursor.execute(""" 
    CREATE TABLE IF NOT EXISTS salary_2 ( 
        empid INT, 
        firstname VARCHAR(50), 
        lastname VARCHAR(50), 
        email VARCHAR(100), 
        phone VARCHAR(20), 
        hire_date DATE, 
        job_id VARCHAR(20), 
        salary INT 
    ) 
""") 

In [36]:

# Suppose 'df' is your DataFrame
for index, row in df.iterrows():
    sql = """ 
        INSERT INTO salary_2 (empid, firstname, lastname, email, phone, 
        hire_date, job_id, salary) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s) 
        ON DUPLICATE KEY UPDATE 
            firstname=VALUES(firstname), 
            lastname=VALUES(lastname), 
            email=VALUES(email), 
            phone=VALUES(phone), 
            hire_date=VALUES(hire_date), 
            job_id=VALUES(job_id), 
            salary=VALUES(salary) 
    """ 
    values = (
        int(row['employee_id']),
        row['first_name'],
        row['last_name'],
        row['email'],
        row['phone_number'],
        row['hire_date'].date() if hasattr(row['hire_date'], 'date') else row['hire_date'],
        row['job_id'],
        int(row['salary'])
    )
    cursor.execute(sql, values)


In [37]:
mydb.commit() 

In [38]:
df = pd.read_sql("SELECT * FROM salary_2", mydb)
print(df)

    empid firstname    lastname     email         phone   hire_date    job_id  \
0     198    Donald    OConnell  DOCONNEL  650.507.9833  2007-06-21  SH_CLERK   
1     199   Douglas       Grant    DGRANT  650.507.9844  2008-01-13  SH_CLERK   
2     200  Jennifer      Whalen   JWHALEN  515.123.4444  2003-09-17   AD_ASST   
3     201   Michael   Hartstein  MHARTSTE  515.123.5555  2004-02-17    MK_MAN   
4     202       Pat         Fay      PFAY  603.123.6666  2005-08-17    MK_REP   
..    ...       ...         ...       ...           ...         ...       ...   
95    136     Hazel  Philtanker  HPHILTAN  650.127.1634  2008-02-06  ST_CLERK   
96    137    Renske      Ladwig   RLADWIG  650.121.1234  2003-07-14  ST_CLERK   
97    138   Stephen      Stiles   SSTILES  650.121.2034  2005-10-26  ST_CLERK   
98    139      John         Seo      JSEO  650.121.2019  2006-02-12  ST_CLERK   
99    140    Joshua       Patel    JPATEL  650.121.1834  2006-04-06  ST_CLERK   

    salary  
0     2600  
1

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_27832\308015245.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM salary_2", mydb)


In [39]:
cursor.close() 
mydb.close() 
logging.error("Something went wrong", exc_info=True) 
print("ETL process completed successfully.")

ETL process completed successfully.


In [40]:
import os 
print(os.getcwd())

C:\Users\ADMIN


In [41]:
!jupyter nbconvert --to script ETLprocess.ipynb

[NbConvertApp] Converting notebook ETLprocess.ipynb to script
[NbConvertApp] Writing 3015 bytes to ETLprocess.py
